# Definimos la arquitectura a utilizar por el algoritmo 

In [6]:
IMAGE_SIZE=224
ARCHITECTURE="mobilenet_0.50_%s" %(IMAGE_SIZE) 
print('ARCHITECTURE:', ARCHITECTURE)

ARCHITECTURE: mobilenet_0.50_224


# Ejecutamos el algoritmo de entrenamiento de tensorflow.
### Parámetros importantes
* architecture: Arquitectura a utilizar para el entrenamiento.
* image_dir: carpeta donde se encuentran las imágenes a clasificar

In [14]:
root_dir = "tf_files/"
bottleneck_dir = root_dir + "bottlenecks/"
model_dir = root_dir + "models/"
output_graph = root_dir + "retrained_graph.pb"
output_labels = root_dir + "retrained_labels.txt"
summaries_dir = root_dir + "training_summaries/"
image_dir = "data/"

In [15]:
%run -i "retrain.py" \
   --bottleneck_dir="{bottleneck_dir}" --how_many_training_steps=500 \
   --model_dir="{model_dir}" --summaries_dir="{summaries_dir}" \
   --output_graph="{output_graph}"  --output_labels="{output_labels}" --architecture="{ARCHITECTURE}"  \
   --image_dir="{image_dir}"

INFO:tensorflow:Looking for images in 'maradona'
INFO:tensorflow:Looking for images in 'messi'
INFO:tensorflow:2018-06-08 16:53:11.353803: Step 0: Train accuracy = 73.0%
INFO:tensorflow:2018-06-08 16:53:11.354516: Step 0: Cross entropy = 0.698512
INFO:tensorflow:2018-06-08 16:53:11.546299: Step 0: Validation accuracy = 62.0% (N=100)
INFO:tensorflow:2018-06-08 16:53:11.951569: Step 10: Train accuracy = 88.0%
INFO:tensorflow:2018-06-08 16:53:11.952164: Step 10: Cross entropy = 0.472082
INFO:tensorflow:2018-06-08 16:53:12.003499: Step 10: Validation accuracy = 100.0% (N=100)
INFO:tensorflow:2018-06-08 16:53:12.398619: Step 20: Train accuracy = 100.0%
INFO:tensorflow:2018-06-08 16:53:12.399337: Step 20: Cross entropy = 0.022378
INFO:tensorflow:2018-06-08 16:53:12.452460: Step 20: Validation accuracy = 91.0% (N=100)
INFO:tensorflow:2018-06-08 16:53:12.860946: Step 30: Train accuracy = 100.0%
INFO:tensorflow:2018-06-08 16:53:12.861698: Step 30: Cross entropy = 0.008341
INFO:tensorflow:2018-0

INFO:tensorflow:2018-06-08 16:53:27.092254: Step 330: Train accuracy = 100.0%
INFO:tensorflow:2018-06-08 16:53:27.092858: Step 330: Cross entropy = 0.001235
INFO:tensorflow:2018-06-08 16:53:27.145512: Step 330: Validation accuracy = 84.0% (N=100)
INFO:tensorflow:2018-06-08 16:53:27.670501: Step 340: Train accuracy = 100.0%
INFO:tensorflow:2018-06-08 16:53:27.671318: Step 340: Cross entropy = 0.001414
INFO:tensorflow:2018-06-08 16:53:27.723241: Step 340: Validation accuracy = 90.0% (N=100)
INFO:tensorflow:2018-06-08 16:53:28.122299: Step 350: Train accuracy = 100.0%
INFO:tensorflow:2018-06-08 16:53:28.122959: Step 350: Cross entropy = 0.001294
INFO:tensorflow:2018-06-08 16:53:28.167921: Step 350: Validation accuracy = 87.0% (N=100)
INFO:tensorflow:2018-06-08 16:53:28.664035: Step 360: Train accuracy = 100.0%
INFO:tensorflow:2018-06-08 16:53:28.664890: Step 360: Cross entropy = 0.001078
INFO:tensorflow:2018-06-08 16:53:28.718967: Step 360: Validation accuracy = 84.0% (N=100)
INFO:tensorf

Testeamos el entrenamiento. En este caso esperamos que infiera que el personaje de la imagen sea Maradona.

In [16]:
%run -i label_image.py --graph=tf_files/retrained_graph.pb --labels=tf_files/retrained_labels.txt --image=data/maradona.jpg


Evaluation time (1-image): 0.248s

maradona (score=0.99979)
messi (score=0.00021)


Luego del entrenamiento, convertiremos el grafo generado a un modelo CoreML que puede correr en dispositivos con IOS.

In [17]:
import tfcoreml as tf_converter


In [18]:
model_name = 'maradona.mlmodel'

tf_converter.convert(tf_model_path = "{output_graph}",
                     mlmodel_path = model_name,
                     image_input_names = 'input:0',
                     class_labels = "{retrained_labels}",
                     output_feature_names = ['final_result:0'],
                     red_bias = -1,
                     green_bias = -1,
                     blue_bias = -1,
                     image_scale = 2.0/255.0)

Shapes not found for 4 tensors. Executing graph to determine shapes. 
Automatic shape interpretation succeeded for input blob input:0
1/560: Analysing op name: MobilenetV1/Predictions/Reshape/shape ( type:  Const )
2/560: Analysing op name: MobilenetV1/Logits/Conv2d_1c_1x1/biases ( type:  Const )
3/560: Analysing op name: MobilenetV1/Logits/Conv2d_1c_1x1/biases/read ( type:  Identity )
4/560: Analysing op name: MobilenetV1/Logits/Conv2d_1c_1x1/weights ( type:  Const )
5/560: Analysing op name: MobilenetV1/Logits/Conv2d_1c_1x1/weights/read ( type:  Identity )
6/560: Analysing op name: MobilenetV1/MobilenetV1/Conv2d_13_pointwise/BatchNorm/batchnorm/add/y ( type:  Const )
7/560: Analysing op name: MobilenetV1/Conv2d_13_pointwise/BatchNorm/moving_variance ( type:  Const )
8/560: Analysing op name: MobilenetV1/Conv2d_13_pointwise/BatchNorm/moving_variance/read ( type:  Identity )
9/560: Analysing op name: MobilenetV1/MobilenetV1/Conv2d_13_pointwise/BatchNorm/batchnorm/add ( type:  Add )
10/

492/560: Analysing op name: MobilenetV1/MobilenetV1/Conv2d_7_depthwise/BatchNorm/batchnorm/mul_1 ( type:  Mul )
493/560: Analysing op name: MobilenetV1/MobilenetV1/Conv2d_7_depthwise/BatchNorm/batchnorm/add_1 ( type:  Add )
494/560: Analysing op name: MobilenetV1/MobilenetV1/Conv2d_7_depthwise/Relu6 ( type:  Relu6 )
495/560: Analysing op name: MobilenetV1/MobilenetV1/Conv2d_7_pointwise/convolution ( type:  Conv2D )
496/560: Analysing op name: MobilenetV1/MobilenetV1/Conv2d_7_pointwise/BatchNorm/batchnorm/mul_1 ( type:  Mul )
497/560: Analysing op name: MobilenetV1/MobilenetV1/Conv2d_7_pointwise/BatchNorm/batchnorm/add_1 ( type:  Add )
498/560: Analysing op name: MobilenetV1/MobilenetV1/Conv2d_7_pointwise/Relu6 ( type:  Relu6 )
499/560: Analysing op name: MobilenetV1/MobilenetV1/Conv2d_8_depthwise/depthwise ( type:  DepthwiseConv2dNative )
500/560: Analysing op name: MobilenetV1/MobilenetV1/Conv2d_8_depthwise/BatchNorm/batchnorm/mul_1 ( type:  Mul )
501/560: Analysing op name: Mobilenet

input {
  name: "input__0"
  type {
    imageType {
      width: 224
      height: 224
      colorSpace: RGB
    }
  }
}
output {
  name: "final_result__0"
  type {
    dictionaryType {
      stringKeyType {
      }
    }
  }
}
output {
  name: "classLabel"
  type {
    stringType {
    }
  }
}
predictedFeatureName: "classLabel"
predictedProbabilitiesName: "final_result__0"

Done!

In [ ]:
!tensorboard --logdir="{training_summaries}"

2018-06-08 16:53:55.513788: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
TensorBoard 1.8.0 at http://Martins-MacBook-Pro.local:6006 (Press CTRL+C to quit)
